# Collect data for github action workflow runs

In this notebook, we collect historical test data like the test duration values from running workflows on Github using the GitHub API

## Collect data for selected workflow runs of a repository

From historical test workflow runs, want to extract
- time durations
- workflow run status & conclusion

We can get workflow IDs of the test that we are interested in from https://api.github.com/repos/{ORG}/{REPO}/actions/workflows 

In [ ]:
from dotenv import find_dotenv, load_dotenv
import os
import json
import subprocess
from datetime import datetime
from subprocess import PIPE
import pandas as pd
from sklearn.model_selection import train_test_split

pd.options.mode.chained_assignment = None

import warnings

warnings.filterwarnings("ignore")

In [ ]:
load_dotenv(find_dotenv(), override=True)
TOKEN = os.getenv("ACCESS_TOKEN")
ORG = os.getenv("ORG")
REPO = os.getenv("REPO")

In [ ]:
# mode can be "check or "workflow" depending on whether you want to collect checks data or workflows data
MODE = "workflow"

* Workflows: For example, lets collect data for the test ID 28698040 for the workflow runs in the repository `oss-aspen/8Knot` https://api.github.com/repos/oss-aspen/8Knot/actions/workflows

* Checks: Get commits for a repo and get check runs for each commit

In [ ]:
def get_page_numbers(test_id):
    """
    Get the total count of tests.
    Find the pages on github-actions.
    """
    command = """curl \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/actions/workflows/{}/runs?""".format(TOKEN, ORG, REPO, test_id)
    args = []
    args.append(command)
    output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
    output = json.loads(output.stdout)
    print(command)
#     total_count = output['total_count']
#     page_numbers = int(total_count/30) # by default number of tests on one page is 30
#     return page_numbers
    return command

In [ ]:
def get_workflow_runs(test_id, page_numbers):
    """
    This function takes test_id and number of pages of workflow runs as input.
    Interacts with github api and collects the data for the tests with the specified id.
    Outputs the data frame with test data.
    """
    for p in range(1,page_numbers+1):
        command = """curl \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/actions/workflows/{}/runs?page={}""".format(TOKEN, ORG, REPO, test_id, p)
        args = []
        args.append(command)

        output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
        output = json.loads(output.stdout)

        if p==1:
            df = pd.json_normalize(output['workflow_runs'])
        else:
            df2 = pd.json_normalize(output['workflow_runs'])
            df = pd.concat([df, df2], axis=0)
    return df

In [ ]:
def get_check_runs(commits):
    """
    This function takes test_id and number of pages of workflow runs as input.
    Interacts with github api and collects the data for the tests with the specified id.
    Outputs the data frame with test data.
    """
    appended_data = []
    for commit in commits:
        command = """curl -L \
          -H "Accept: application/vnd.github+json" \
          -H "Authorization: Bearer {}"\
          -H "X-GitHub-Api-Version: 2022-11-28" \
          https://api.github.com/repos/{}/{}/commits/{}/check-runs""".format(TOKEN, ORG, REPO, commit)
        args = []
        args.append(command)

        output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
        output = json.loads(output.stdout)

        appended_data.append(pd.json_normalize(output['check_runs']))

    df = pd.concat(appended_data, axis=0)

    return df

In [ ]:
def get_commits():
    "This function gets all commit refs for a github repo"

    command =  """curl -L \
      -H "Accept: application/vnd.github+json" \
      -H "Authorization: Bearer {}"\
      -H "X-GitHub-Api-Version: 2022-11-28" \
      https://api.github.com/repos/{}/{}/commits""".format(TOKEN, ORG, REPO)
    args = []
    args.append(command)

    output = subprocess.run(args, shell=True, check=True, stdout=PIPE, stderr=PIPE)
    output = json.loads(output.stdout)

    df = pd.json_normalize(output)
    return df

In [ ]:
if MODE == "check":
    # get checks
    df_commits = get_commits()
    commit_ids = list(df_commits['sha'])
    checks_df = get_check_runs(commit_ids)
    test_df = checks_df[['started_at', 'completed_at', 'id', 'status', 'conclusion', 'external_id', 'name']]
    test_df = test_df.rename(columns={'external_id': 'test_id', 'id': 'run_id'})
    test_df['run_duration'] = test_df.apply(lambda x: (datetime.strptime(x['completed_at'], "%Y-%m-%dT%H:%M:%SZ") - \
                                           datetime.strptime(x['started_at'], "%Y-%m-%dT%H:%M:%SZ")).total_seconds(), axis = 1)

elif MODE == "workflow":
    # get workflows
    test_id = "28698040" # Pre-commit test
    page_numbers = get_page_numbers(test_id)
    print(page_numbers)
#     workflow_df = get_workflow_runs(test_id, page_numbers)
#     test_df = workflow_df[['created_at', 'updated_at', 'id', 'status', 'conclusion']]
#     test_df = test_df.rename(columns={'id': 'run_id'})
#     test_df['run_duration'] = test_df.apply(lambda x: (datetime.strptime(x['updated_at'], "%Y-%m-%dT%H:%M:%SZ") - \
#                                            datetime.strptime(x['created_at'], "%Y-%m-%dT%H:%M:%SZ")).total_seconds(), axis = 1)
#     test_df['test_id'] = test_id

In [ ]:
# test_df.head(10)

In [ ]:
# test_df.shape

In [ ]:
# # generating passing and failing dfs which are neccesary for computing fit distributions
# passing_df = test_df[test_df['conclusion'] == 'success'] 
# failures_df = test_df[test_df['conclusion'] == 'failure'] 

In [ ]:
# passing_df.shape

In [ ]:
# failures_df.shape

In [ ]:
# Split the data into train and test.
# passing_train, passing_test = train_test_split(passing_df, test_size=0.2, random_state=25)

In [ ]:
# Split the data into train and test
# failing_train, failing_test = train_test_split(failures_df, test_size=0.2, random_state=25)

In [ ]:
# passing_train.shape

In [ ]:
# passing_test.shape

In [ ]:
# failing_train.shape

In [ ]:
# failing_test.shape

In [ ]:
# passing_train.to_csv("../data/processed/passing_train.csv")
# failing_train.to_csv("../data/processed/failing_train.csv")
# passing_test.to_csv("../data/processed/passing_test.csv")
# failing_test.to_csv("../data/processed/failing_test.csv")

## Conclusion

In this notebook, we interact with the github api to collect the data for all workflow runs. In future work, we will look into using this data to perform statistical tests using OSP model.